In [1]:
import numpy as np

# --- 1. 定义物理和数值参数 (同 (b) 问) ---
L = 1.0         # 板的长度 (m)
W = 1.0         # 板的宽度 (m)
k = 10.0        # 热导率 (W/(m·K))
q_s_prime = 2000.0  # 热通量 (W/m^2)
T1 = 30.0       # 恒定边界温度 (°C)

# 数值参数 (用于 FDM)
Nx = 51         # x 方向的网格节点数
Ny = 51         # y 方向的网格节点数
dx = L / (Nx - 1)
dy = W / (Ny - 1)
tolerance = 1e-6  # 收敛判据
max_iter = 10000    # 最大迭代次数

# --- 2. 执行 (b) 问的 FDM 计算 ---
# (这部分代码与 (b) 问相同，用于获取 T 矩阵)

print("正在执行 (b) 问的 FDM 计算...")
T = np.full((Ny, Nx), T1) # 初始猜测值

# 应用 Dirichlet 边界条件
T[0, :] = T1   # 下边界 (j=0)
T[:, 0] = T1   # 左边界 (i=0)
T[:, -1] = T1  # 右边界 (i=Nx-1)

# 预计算通量项
flux_term = (2 * dy * q_s_prime) / k

for iteration in range(max_iter):
    T_old = T.copy()
    
    # 迭代内部节点
    for j in range(1, Ny - 1):
        for i in range(1, Nx - 1):
            T[j, i] = 0.25 * (T[j, i+1] + T[j, i-1] + T[j+1, i] + T[j-1, i])
            
    # 处理上边界 (j=Ny-1)
    j = Ny - 1
    for i in range(1, Nx - 1):
        T[j, i] = 0.25 * (T[j, i+1] + T[j, i-1] + 2 * T[j-1, i] + flux_term)

    # 检查收敛性
    diff = np.max(np.abs(T - T_old))
    if diff < tolerance:
        print(f"FDM 在 {iteration + 1} 次迭代后收敛。")
        break
else:
    print(f"FDM 达到最大迭代次数 {max_iter}。")

# --- 3. 求解 (c) 问：计算下表面传热速率 ---
print("\n--- (c) 问：计算下表面传热速率 ---")

# (c) 问的计算思路:
# 1. 传热速率 q' = ∫[0, L] q_y''(x) dx
# 2. 离开下表面的热通量 q_y''(x) = -k * (∂T/∂y) |_(y=0)
# 3. FDM 近似 (前向差分):
#    (∂T/∂y) |_(y=0, x=i) ≈ (T[1, i] - T[0, i]) / dy
# 4. 数值积分: 使用梯形法则 (np.trapz) 沿 x 轴积分

# 获取下边界 (j=0) 上的温度 (即 T1)
T_bottom = T[0, :]  # 这是 y=0 处的温度行

# 获取下边界上方的第一行内部节点 (j=1) 的温度
T_first_internal = T[1, :] # 这是 y=dy 处的温度行

# 1. 计算 y 方向的温度梯度 (∂T/∂y)
dT_dy = (T_first_internal - T_bottom) / dy

# 2. 计算热通量 q_y''(x) (W/m^2)
q_flux_array = k * dT_dy  # 注意：我们计算的是流出板的热量，故符号为正

# 3. 沿 x 轴 (从 0 到 L) 积分热通量，得到总传热速率 (W/m)
x_coords = np.linspace(0, L, Nx)
q_total_lower_fdm = np.trapz(q_flux_array, x_coords)

print(f"FDM 计算的下表面传热速率: {q_total_lower_fdm:.4f} W/m")


# --- 4. (验证) 使用解析解计算 (c) 问 ---
# 在 (a) 问的分析中，我们推导了下表面的传热速率解析解：
# q'_lower,out = (8 * q_s'' * L / (pi^2)) * Σ [1 / (n^2 * cosh(n*pi*W/L))] (n=1,3,5...)

def q_analytical(N_terms=50):
    q_sum = 0
    C = (8 * q_s_prime * L) / (np.pi**2) # L=1
    for n_odd in range(1, 2 * N_terms + 1, 2):
        n_pi_W_L = n_odd * np.pi * W / L # L=1, W=1
        term = 1 / (n_odd**2 * np.cosh(n_pi_W_L))
        q_sum += term
    return C * q_sum

q_total_lower_ana = q_analytical(N_terms=50)
print(f"解析解计算的下表面传热速率: {q_total_lower_ana:.4f} W/m")

# 计算 FDM 解与解析解的误差
error = np.abs(q_total_lower_fdm - q_total_lower_ana)
print(f"计算误差: {error:.6f} W/m")

正在执行 (b) 问的 FDM 计算...
FDM 在 4674 次迭代后收敛。

--- (c) 问：计算下表面传热速率 ---
FDM 计算的下表面传热速率: 139.9277 W/m
解析解计算的下表面传热速率: 139.8795 W/m
计算误差: 0.048262 W/m
